In [1]:
%load_ext autoreload
%autoreload 2

import os
import json
import sqlite3
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from tqdm import tqdm

pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)

**Load in data**

In [2]:
all_cols = [
    "gcs_mn_all", "gcs_mn_se_all", "tot_asmt_all",
    "gcs_mn_asn", "gcs_mn_se_asn", "tot_asmt_asn",
    "gcs_mn_blk", "gcs_mn_se_blk", "tot_asmt_blk",
    "gcs_mn_ecd", "gcs_mn_se_ecd", "tot_asmt_ecd",
    "gcs_mn_fem", "gcs_mn_se_fem", "tot_asmt_fem",
    "gcs_mn_hsp", "gcs_mn_se_hsp", "tot_asmt_hsp",
    "gcs_mn_mal", "gcs_mn_se_mal", "tot_asmt_mal",
    "gcs_mn_mfg", "gcs_mn_se_mfg", "tot_asmt_mfg",
    "gcs_mn_nam", "gcs_mn_se_nam", "tot_asmt_nam",
    "gcs_mn_nec", "gcs_mn_se_nec", "tot_asmt_nec",
    "gcs_mn_neg", "gcs_mn_se_neg", "tot_asmt_neg",
    "gcs_mn_wag", "gcs_mn_se_wag", "tot_asmt_wag",
    "gcs_mn_wbg", "gcs_mn_se_wbg", "tot_asmt_wbg",
    "gcs_mn_whg", "gcs_mn_se_whg", "tot_asmt_whg",
    "gcs_mn_wht", "gcs_mn_se_wht", "tot_asmt_wht",
    "gcs_mn_wng", "gcs_mn_se_wng", "tot_asmt_wng",
]

race_cols = [
    "gcs_mn_all", "gcs_mn_se_all", "tot_asmt_all",
    "gcs_mn_asn", "gcs_mn_se_asn", "tot_asmt_asn",
    "gcs_mn_blk", "gcs_mn_se_blk", "tot_asmt_blk",
    "gcs_mn_hsp", "gcs_mn_se_hsp", "tot_asmt_hsp",
    "gcs_mn_nam", "gcs_mn_se_nam", "tot_asmt_nam",
]

gender_cols = [
    "gcs_mn_all", "gcs_mn_se_all", "tot_asmt_all",
    "gcs_mn_fem", "gcs_mn_se_fem", "tot_asmt_fem",
    "gcs_mn_mal", "gcs_mn_se_mal", "tot_asmt_mal",
]

In [ ]:
if not os.path.exists("./data/data.db"):
    os.system("python build_db.py")

conn = sqlite3.connect("./data/data.db")
c = conn.cursor()


COLS_EXCLUDE = set([
    "StateAbbr",
    "StateDesc",
    "CountyName",
    "CountyFIPS",
    "TractFIPS",
    "fips",
    "stateabb",
    "sedaadmin",
    "sedaadminname",
    "TRACT",
    "COUNT"
])

c.execute(f"""
WITH seda_tracts AS (
    SELECT *
    FROM seda s
    JOIN nces n
    ON s.sedaadmin = n.LEAID
    WHERE s.year = 2019
)

, food_atlas_tracts AS (
    SELECT *
    FROM food f
    JOIN seda_tracts st
    ON st.TRACT = f.CensusTract
)

, cdc_tracts AS (
    SELECT *
    FROM cdc c
    JOIN food_atlas_tracts ft
    ON ft.TRACT = c.TractFIPS
)

SELECT *
FROM cdc_tracts;
""")

rows = c.fetchall()
print(len(rows))
columns = [col[0] for col in c.description]

## Write the rows manually into a CSV file without pandas
with open("data/full_joined_table.csv", "w") as f:
    to_exclude = set()
    # race_cols = []
    # for i, col in enumerate(columns):
    #     if (col.startswith("gcs") or col.startswith("tot")) and col not in race_cols:
    #         to_exclude.add(i)

    f.write(",".join(columns) + "\n")
    for row in tqdm(rows):
        f.write(",".join([str(x) for i, x in enumerate(row) if i not in to_exclude]) + "\n")


995894


100%|██████████| 995894/995894 [00:13<00:00, 74453.65it/s]


In [ ]:
df = pd.read_csv("full_joined_table.csv")
df